In [ ]:
import requests
from bs4 import BeautifulSoup
import wget
import os

In [ ]:

link = "https://www.conab.gov.br/info-agro/safras/serie-historica-das-safras/itemlist/category/913-trigo"
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:128.0) Gecko/20100101 Firefox/128.0"}
requisicao = requests.get(link, headers=headers)
# print(requisicao)
site = BeautifulSoup(requisicao.text, "html.parser")
# print(site.prettify())
# pesquisa = site.find_all("a")
# print(pesquisa[1])
pesquisa = site.find("a", title="TrigoSerieHist.xls")
link_part = pesquisa["href"]
site = "https://www.conab.gov.br"
link = site + link_part

# Deletar o arquivo antes de fazer o download
try:
    os.remove("arquivos/trigo.xls")
except OSError as e:
    print(f"Error:{ e.strerror}")

# Fazer o download do arquivo
wget.download(link, "arquivos/trigo.xls")

Tratar arquivos com pandas

Bibliotecas utilizadas:
    pip install pandas
    pip install openpyxl
    pip install xlrd

In [ ]:
import pandas as pd
import openpyxl

In [ ]:
try:
    df = pd.read_excel("arquivos/trigo.xls",sheet_name='Produção')
except OSError as e:
    print(f"Error:{ e.strerror}")

df = df.loc[4:38] # selecionando as linhas no intervalo de 4 até 38
df.columns = df.iloc[0] # definindo o primeira linha como cabeçalho
df = df.drop(4,axis=0) #excluindo a linha 4
df = df.rename(columns={'REGIÃO/UF': 'UF'})
df.loc[df['UF'].str.len() > 2, 'REGIAO'] =  df['UF']
df['REGIAO'] = df['REGIAO'].ffill()
df = df.melt(id_vars=['UF','REGIAO'], var_name='ANO', value_name='TOTAL')
df = df[df['UF'].str.len() == 2]
df = df[df['TOTAL'] > 0]
df.reset_index(drop=True, inplace=True)
df.to_excel("arquivos/trigo.xlsx", index=False, sheet_name='Produção')
display(df)

#display(df.describe())